* 因子分析のloadingsを使用してグルーピングしてみる
* 排他ではなく、重複を許してみる
* Ver5でGaussianMixture n=3

In [ ]:
!pip install japanize-matplotlib

In [ ]:
# !pip install git+https://github.com/darecophoenixx/wordroid.sblo.jp

In [ ]:
%matplotlib inline
from IPython.display import SVG, Image
from keras.utils.vis_utils import model_to_dot

import random, re

import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture

import matplotlib.pyplot as plt
import seaborn as sns

import japanize_matplotlib
plt.rcParams['font.family']

In [ ]:
def hexbin(x, y, color, **kwargs):
    cmap = sns.light_palette(color, as_cmap=True)
    plt.hexbin(x, y, cmap=cmap, **kwargs)
def scatter(x, y, color, **kwargs):
    plt.scatter(x, y, marker='.')

In [ ]:
img_cnt = 0
def save_img():
    global img_cnt
    img_cnt += 1
    img_name = 'img_{0:03}.png'.format(img_cnt)
    print(img_name)
    plt.savefig(img_name)

# データ 読み込み

In [ ]:
'''元の０/１データ'''
df = pd.read_csv('../input/sample-data-wordanddoc2vec/actual_survey_data_jp.csv', index_col=0)
print(df.shape)
df.head()

In [ ]:
'''
因子負荷量
psych::faの方
'''
loadings = pd.read_csv('../input/apply-factanal-to-asd-calc-cor-from-col/res2_fa.csv', index_col=0)
print(loadings.shape)
loadings.head()

# 疑似スコアを計算する

In [ ]:
'''疑似スコアを計算する'''
scores = df.values.dot(loadings.values)
print(scores.shape)

In [ ]:
df_scores = pd.DataFrame(scores, index=df.index, columns=loadings.columns)
print(df_scores.shape)
df_scores.head()

In [ ]:
plt.hist(scores[0,:])

In [ ]:
sns.pairplot(pd.DataFrame(scores[:,:3]))

In [ ]:
sns.kdeplot(data=df_scores, x='MR1')

In [ ]:
sns.kdeplot(data=df_scores, x='MR8')

In [ ]:
sns.kdeplot(data=df_scores, x='MR2')

In [ ]:
gm = GaussianMixture(n_components=3, n_init=5)

* 混合正規分布で各変数を分類する
* ３分類し、一番値の高いクラスを採用する

In [ ]:
for tgt_col in loadings.columns.values:
    print('=== tgt_col ->', tgt_col)
    gm.fit(df_scores[[tgt_col]])
    print(gm.means_)
    print('gm.means_.argmax() ->', gm.means_.argmax())
    idx = (gm.predict(df_scores[[tgt_col]]) == gm.means_.argmax())
    print(idx.sum())
    pred = np.zeros((df_scores.shape[0],), dtype=int)
    pred[idx] = 1
    df_scores[tgt_col+'cls'] = pred
sns.kdeplot(data=df_scores, x=tgt_col, hue=tgt_col+'cls')

In [ ]:
df_scores

In [ ]:
df_scores.to_csv('df_scores.csv')